In [1]:
#Importa os pacotes necessários

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import datetime as dt
plt.style.use('dark_background')

#### Dicionário de Variáveis utilizadas
|Chave|Descrição|Função|
|-----|-----|-----|
|Ano| Ano de referência|Identificação|
|Trimestre| Trimestre de referência|Identificação|
|UF| Estado|Identificação|
|Capital| Capital da UF|Identificação|
|UPA| Unidade Primária de Amostragem (UF(2) + número squencial(6) + DV(1))|
|V1008| Número do Domicílio|Identificação|
|Estrato||
|V1014| Painel - Grupo de Amostra|
|V1016| Número de entrevista (1 a 5)|
|V2003| Número de ordem|
|V2007| Sexo|Identificação|
|V2008| Dia Nascimento|
|V20081| Mês nascimento|
|V20082| Ano nascimento|
|V2010| Raça|Identificação|
|V4041| Código de Ocupação -> cruzar com a tabela COD 2010|Medida|
|V4044| Código da Principal atividade da empresa -> cruzar a CNAE-Domicilar|Medida|
|V1028| Peso do domicílio e das pessoas sem calibração|Expansão|
|VD1029| Projeção da população|Expansão|

Chave para pessoa: UPA + V1008 + V1014  + V2003 + V2007 + V2008 + V20081 + V20082

**PARA ESTIMAÇÃO DA POPULAÇÃO, DEVE-SE USAR AS VARIÁVEIS: V1033 e posest.**

#### Importação e tratamento dos microdados

Importação dos microdados

In [2]:
#determina qual diretório usar para cada sistema operacional.
import os
if os.name == 'nt': #windows
    diretorio = 'C:\\Users\\est.matheus.brasil\\Documents\\Microdados PNAD Contínua\\'
    enconding = 'latin_1'
else: #linux(ou mac)
    diretorio = '/home/matheus/Microdados PNAD/'
    enconding = 'utf_8_sig'

#Leitura de todas as bases com as variáveis selecionadas e criação de variáveis dinamicamente.

colunas_selecionadas = ['Ano', 'Trimestre', 'UPA', 'UF', 'V1008', 'V1014', 'V2007', 'V2008', 'V20081', 'V20082', 'V2010', 'V4041', 'V4044', 'V1028', 'V1029', 'V1033', 'posest', 'posest_sxi']

for ano in range(2019, 2023):
    for tri in range(1, 5):
        if ano == 2019  and tri > 3:
            print(ano, tri)
            globals()[f'pnad_{ano}_{tri}'] = pd.read_feather(fr'{diretorio}{ano}_{tri}.feather', columns=colunas_selecionadas)
        if 2019 < ano < 2022:
            print(ano, tri)
            globals()[f'pnad_{ano}_{tri}'] = pd.read_feather(fr'{diretorio}{ano}_{tri}.feather', columns=colunas_selecionadas)
        if ano == 2022 and tri < 3:
            print(ano, tri)
            globals()[f'pnad_{ano}_{tri}'] = pd.read_feather(fr'{diretorio}{ano}_{tri}.feather', columns=colunas_selecionadas)

2019 4
2020 1
2020 2
2020 3
2020 4
2021 1
2021 2
2021 3
2021 4
2022 1
2022 2


In [3]:
#Conversão das variável V2007(sexo) para inteira para criar ID do indivíduo. (Homem = 0, Mulher = 1)
for ano in range(2019, 2023):
    for tri in range(1, 5):
        if ano == 2019  and tri > 3:
            globals()[f'pnad_{ano}_{tri}']['V2007'] = np.where(globals()[f'pnad_{ano}_{tri}']['V2007'] == 'Homem', 0, 1)
        if 2019 < ano < 2022:
            globals()[f'pnad_{ano}_{tri}']['V2007'] = np.where(globals()[f'pnad_{ano}_{tri}']['V2007'] == 'Homem', 0, 1)
        if ano == 2022 and tri < 3:
            globals()[f'pnad_{ano}_{tri}']['V2007'] = np.where(globals()[f'pnad_{ano}_{tri}']['V2007'] == 'Homem', 0, 1)


In [4]:
#Organização das variáveis e criação da chave de ID do indivíduo em cada dataframe.

for ano in range(2019, 2023):
    for tri in range(1, 5):
        if ano == 2019  and tri > 3:
            globals()[f'pnad_{ano}_{tri}_id_cnae'] = pd.DataFrame(
                                                        {'Ano':globals()[f'pnad_{ano}_{tri}']['Ano'], 
                                                        'Trimestre':globals()[f'pnad_{ano}_{tri}']['Trimestre'],
                                                        'UPA':globals()[f'pnad_{ano}_{tri}']['UPA'],
                                                        'UF':globals()[f'pnad_{ano}_{tri}']['UF'],
                                                        'V2007':globals()[f'pnad_{ano}_{tri}']['V2007'],  
                                                        'ID': globals()[f'pnad_{ano}_{tri}']['UPA'] + 
                                                              globals()[f'pnad_{ano}_{tri}']['V1008'] + 
                                                              globals()[f'pnad_{ano}_{tri}']['V1014'] + 
                                                              globals()[f'pnad_{ano}_{tri}']['V2007'] +
                                                              globals()[f'pnad_{ano}_{tri}']['V2008'] +
                                                              globals()[f'pnad_{ano}_{tri}']['V20081'] +
                                                              globals()[f'pnad_{ano}_{tri}']['V20082'],
                                                        'V2010': globals()[f'pnad_{ano}_{tri}']['V2010'],
                                                        'V4041': globals()[f'pnad_{ano}_{tri}']['V4041'],
                                                        'V4044': globals()[f'pnad_{ano}_{tri}']['V4044'],
                                                        'V1028': globals()[f'pnad_{ano}_{tri}']['V1028'],
                                                        'V1029': globals()[f'pnad_{ano}_{tri}']['V1029'],
                                                        'V1033': globals()[f'pnad_{ano}_{tri}']['V1033'],
                                                        'posest': globals()[f'pnad_{ano}_{tri}']['posest'],
                                                        'posest_sxi': globals()[f'pnad_{ano}_{tri}']['posest_sxi']
                                                      })
        if 2019 < ano < 2022:
            globals()[f'pnad_{ano}_{tri}_id_cnae'] = pd.DataFrame(
                                                        {'Ano':globals()[f'pnad_{ano}_{tri}']['Ano'], 
                                                        'Trimestre':globals()[f'pnad_{ano}_{tri}']['Trimestre'],
                                                        'UPA':globals()[f'pnad_{ano}_{tri}']['UPA'],
                                                        'UF':globals()[f'pnad_{ano}_{tri}']['UF'],
                                                        'V2007':globals()[f'pnad_{ano}_{tri}']['V2007'],  
                                                        'ID': globals()[f'pnad_{ano}_{tri}']['UPA'] + 
                                                              globals()[f'pnad_{ano}_{tri}']['V1008'] + 
                                                              globals()[f'pnad_{ano}_{tri}']['V1014'] + 
                                                              globals()[f'pnad_{ano}_{tri}']['V2007'] +
                                                              globals()[f'pnad_{ano}_{tri}']['V2008'] +
                                                              globals()[f'pnad_{ano}_{tri}']['V20081'] +
                                                              globals()[f'pnad_{ano}_{tri}']['V20082'],
                                                        'V2010': globals()[f'pnad_{ano}_{tri}']['V2010'],
                                                        'V4041': globals()[f'pnad_{ano}_{tri}']['V4041'],
                                                        'V4044': globals()[f'pnad_{ano}_{tri}']['V4044'],
                                                        'V1028': globals()[f'pnad_{ano}_{tri}']['V1028'],
                                                        'V1029': globals()[f'pnad_{ano}_{tri}']['V1029'],
                                                        'V1033': globals()[f'pnad_{ano}_{tri}']['V1033'],
                                                        'posest': globals()[f'pnad_{ano}_{tri}']['posest'],
                                                        'posest_sxi': globals()[f'pnad_{ano}_{tri}']['posest_sxi']
                                                      })
        if ano == 2022 and tri < 3:
            globals()[f'pnad_{ano}_{tri}_id_cnae'] = pd.DataFrame(
                                                        {'Ano':globals()[f'pnad_{ano}_{tri}']['Ano'], 
                                                        'Trimestre':globals()[f'pnad_{ano}_{tri}']['Trimestre'],
                                                        'UPA':globals()[f'pnad_{ano}_{tri}']['UPA'],
                                                        'UF':globals()[f'pnad_{ano}_{tri}']['UF'],
                                                        'V2007':globals()[f'pnad_{ano}_{tri}']['V2007'],  
                                                        'ID': globals()[f'pnad_{ano}_{tri}']['UPA'] + 
                                                              globals()[f'pnad_{ano}_{tri}']['V1008'] + 
                                                              globals()[f'pnad_{ano}_{tri}']['V1014'] + 
                                                              globals()[f'pnad_{ano}_{tri}']['V2007'] +
                                                              globals()[f'pnad_{ano}_{tri}']['V2008'] +
                                                              globals()[f'pnad_{ano}_{tri}']['V20081'] +
                                                              globals()[f'pnad_{ano}_{tri}']['V20082'],
                                                        'V2010': globals()[f'pnad_{ano}_{tri}']['V2010'],
                                                        'V4041': globals()[f'pnad_{ano}_{tri}']['V4041'],
                                                        'V4044': globals()[f'pnad_{ano}_{tri}']['V4044'],
                                                        'V1028': globals()[f'pnad_{ano}_{tri}']['V1028'],
                                                        'V1029': globals()[f'pnad_{ano}_{tri}']['V1029'],
                                                        'V1033': globals()[f'pnad_{ano}_{tri}']['V1033'],
                                                        'posest': globals()[f'pnad_{ano}_{tri}']['posest'],
                                                        'posest_sxi': globals()[f'pnad_{ano}_{tri}']['posest_sxi']
                                                      })

In [5]:
#Concatena todos os dataframes de cada ano e trimestre em uma unica tabela.

pnadc_completa = pd.concat([pnad_2019_4_id_cnae, 
                            pnad_2020_1_id_cnae, 
                            pnad_2020_2_id_cnae, 
                            pnad_2020_3_id_cnae, 
                            pnad_2020_4_id_cnae, 
                            pnad_2021_1_id_cnae, 
                            pnad_2021_2_id_cnae, 
                            pnad_2021_3_id_cnae, 
                            pnad_2021_4_id_cnae, 
                            pnad_2022_1_id_cnae, 
                            pnad_2022_2_id_cnae])

In [6]:
pnadc_completa

,Ano,Trimestre,UPA,UF,V2007,ID,V2010,V4041,V4044,V1028,V1029,V1033,posest,posest_sxi
0,2019,4,110000016,Rondônia,1,110002003,Parda,NaN,NaN,145.628790,532471,6982307,111,210
1,2019,4,110000016,Rondônia,1,110002034,Parda,NaN,NaN,145.628790,532471,8485527,111,206
2,2019,4,110000016,Rondônia,1,110002051,Parda,NaN,NaN,145.628790,532471,9316281,111,204
3,2019,4,110000016,Rondônia,0,110002077,Parda,NaN,NaN,145.628790,532471,7432741,111,102
4,2019,4,110000016,Rondônia,0,110002012,Parda,NaN,NaN,134.017538,532471,5214819,111,112
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
482113,2022,2,530057544,Distrito Federal,0,530059585,Parda,NaN,NaN,81.626590,3125014,8442398,531,107
482114,2022,2,530057544,Distrito Federal,0,530059543,Preta,NaN,NaN,72.323502,3125014,3560554,531,114
482115,2022,2,530057544,Distrito Federal,0,530059578,Preta,NaN,NaN,72.323502,3125014,8409440,531,108
482116,2022,2,530057544,Distrito Federal,1,530059586,Branca,NaN,NaN,72.323502,3125014,8558427,531,207


In [7]:
# Ordernação da série histórica por ano e trimestre.
pnadc_completa.sort_values(by=['Ano', 'Trimestre'], inplace=True)

In [8]:
#concatena a coluna 'Ano' e 'Trimestre' em uma única coluna.
pnadc_completa['ano_tri'] = pnadc_completa['Ano'].astype(str) + 'Q' + pnadc_completa['Trimestre'].astype(str)

In [9]:
#Conversão da coluna ano_tri para datetime.
pnadc_completa['ano_tri'] = pd.PeriodIndex(pnadc_completa['ano_tri'], freq='Q')

#### Cruzamento da CNAE domiciliar com a base da PNAD  para identificar o setor de atividade econômica.

In [10]:
cod_2010 = pd.read_excel('Anexos/COD 2010.xlsx')

In [11]:
cod_2010.dropna(inplace=True)

In [12]:
cod_2010

,0,Grandes Grupos,Subgrupos Principal,Subgrupos,Grupos de Base,Titulação
3,1111,1,11.0,111.0,1111.0,LEGISLADORES
4,1112,1,11.0,111.0,1112.0,DIRIGENTES SUPERIORES DA ADMINISTRAÇÃO PÚBLICA
5,1113,1,11.0,111.0,1113.0,CHEFES DE PEQUENAS POPULAÇÕES
6,1114,1,11.0,111.0,1114.0,DIRIGENTES DE ORGANIZAÇÕES QUE APRESENTAM UM I...
8,1120,1,11.0,112.0,1120.0,DIRETORES GERAIS E GERENTES GERAIS
...,...,...,...,...,...,...
197,3141,3,31.0,314.0,3141.0,TÉCNICOS E PROFISSIONAIS DE NÍVEL MÉDIO EM CIÊ...
198,3142,3,31.0,314.0,3142.0,TÉCNICOS AGROPECUÁRIOS
199,3143,3,31.0,314.0,3143.0,TÉCNICOS FLORESTAIS
201,3151,3,31.0,315.0,3151.0,OFICIAIS MAQUINISTAS EM NAVEGAÇÃO


In [13]:
#Criar um dicionário com as colunas Grupos de base e Titulação
cod_2010_dict = dict(zip(cod_2010['Grupos de Base'], cod_2010['Titulação']))
cod_2010_dict

{1111.0: 'LEGISLADORES',
 1112.0: 'DIRIGENTES SUPERIORES DA ADMINISTRAÇÃO PÚBLICA',
 1113.0: 'CHEFES DE PEQUENAS POPULAÇÕES',
 1114.0: 'DIRIGENTES DE ORGANIZAÇÕES QUE APRESENTAM UM INTERESSE ESPECIAL',
 1120.0: 'DIRETORES GERAIS E GERENTES GERAIS',
 1211.0: 'DIRIGENTES FINANCEIROS',
 1212.0: 'DIRIGENTES DE RECURSOS HUMANOS',
 1213.0: 'DIRIGENTES DE POLÍTICAS E PLANEJAMENTO',
 1219.0: 'DIRIGENTES DE ADMINISTRAÇÃO E DE SERVIÇOS NÃO CLASSIFICADOS ANTERIORMENTE',
 1221.0: 'DIRIGENTES DE VENDAS E  COMERCIALIZAÇÃO',
 1222.0: 'DIRIGENTES DE PUBLICIDADE E RELAÇÕES PÚBLICAS',
 1223.0: 'DIRIGENTES DE PESQUISA E DESENVOLVIMENTO',
 1311.0: 'DIRIGENTES DE PRODUÇÃO AGROPECUÁRIA E SILVICULTURA',
 1312.0: 'DIRIGENTES DE PRODUÇÃO DA AQUICULTURA E PESCA',
 1321.0: 'DIRIGENTES DE INDÚSTRIA DE TRANSFORMAÇÃO',
 1322.0: 'DIRIGENTES DE EXPLORAÇÕES DE MINERAÇÃO',
 1323.0: 'DIRIGENTES DE EMPRESAS DE CONSTRUÇÃO',
 1324.0: 'DIRIGENTES DE EMPRESAS DE ABASTECIMENTO, DISTRIBUIÇÃO E AFINS',
 1330.0: 'DIRIGENTES DE S

In [14]:
#mapear a coluna V4041 com o dicionário criado.
pnadc_completa['V4041'] = pnadc_completa['V4041'].map(cod_2010_dict)

In [15]:
ocupacao_expandida = pd.crosstab(index=pnadc_completa['ano_tri'],
                            columns=pnadc_completa['V4041'], 
                            values=(pnadc_completa['V1029']*pnadc_completa['posest'])/pnadc_completa['V1029'], 
                            aggfunc='sum')                            

ocupacao_expandida

V4041,ADMINISTRADORES DE SISTEMAS,ADVOGADOS E JURISTAS,AGRÔNOMOS E AFINS,ANALISTAS DE GESTÃO E ADMINISTRAÇÃO,ANALISTAS DE SISTEMAS,ANALISTAS FINANCEIROS,ARQUITETOS DE EDIFICAÇÕES,ARQUITETOS PAISAGISTAS,ARQUIVOLOGISTAS E CURADORES DE MUSEUS,ARTISTAS CRIATIVOS E INTERPRETATIVOS NÃO CLASSIFICADOS ANTERIORMENTE,...,TÉCNICOS AGROPECUÁRIOS,TÉCNICOS E PROFISSIONAIS DE NÍVEL MÉDIO EM CIÊNCIAS BIOLÓGICAS (EXCLUSIVE DA MEDICINA),TÉCNICOS EM CIÊNCIAS FÍSICAS E DA ENGENHARIA NÃO CLASSIFICADOS ANTERIORMENTE,TÉCNICOS EM CIÊNCIAS FÍSICAS E QUÍMICAS,TÉCNICOS EM ELETRÔNICA,TÉCNICOS EM ENGENHARIA CIVIL,TÉCNICOS EM ENGENHARIA MECÂNICA,TÉCNICOS EM QUÍMICA INDUSTRIAL,URBANISTAS E ENGENHEIROS DE TRÂNSITO,VETERINÁRIOS
ano_tri,,,,,,,,,,,,,,,,,,,,,
2019Q4,NaN,39387.0,1082.0,10341.0,2449.0,1327.0,6005.0,NaN,NaN,NaN,...,748.0,502.0,2092.0,NaN,1669.0,948.0,1593.0,NaN,334.0,4990.0
2020Q1,NaN,24880.0,NaN,4046.0,3202.0,582.0,6709.0,NaN,421.0,NaN,...,676.0,251.0,1432.0,NaN,775.0,462.0,1450.0,332.0,NaN,2910.0
2020Q2,NaN,23567.0,171.0,5544.0,1794.0,1408.0,5312.0,314.0,NaN,NaN,...,1380.0,NaN,332.0,NaN,1942.0,804.0,NaN,332.0,NaN,2890.0
2020Q3,NaN,25340.0,171.0,5673.0,3231.0,NaN,3446.0,314.0,862.0,NaN,...,708.0,NaN,997.0,NaN,1242.0,495.0,NaN,NaN,332.0,1918.0
2020Q4,NaN,21706.0,325.0,4937.0,2610.0,893.0,4321.0,314.0,NaN,NaN,...,798.0,NaN,666.0,NaN,1694.0,809.0,354.0,NaN,332.0,2868.0
2021Q1,NaN,17821.0,816.0,1920.0,1774.0,1217.0,3318.0,311.0,NaN,411.0,...,1306.0,NaN,NaN,NaN,1140.0,840.0,871.0,NaN,NaN,2650.0
2021Q2,NaN,18192.0,1644.0,4353.0,1401.0,262.0,3696.0,NaN,NaN,411.0,...,508.0,NaN,NaN,NaN,1411.0,636.0,262.0,414.0,NaN,2784.0
2021Q3,354.0,21871.0,1494.0,5274.0,2378.0,907.0,3854.0,NaN,NaN,411.0,...,358.0,NaN,NaN,124.0,3548.0,354.0,729.0,414.0,NaN,2306.0
2021Q4,151.0,24030.0,1797.0,6892.0,3620.0,648.0,6003.0,NaN,NaN,NaN,...,628.0,NaN,531.0,NaN,1392.0,906.0,1968.0,NaN,NaN,2545.0
